In [78]:
import os
import pandas as pd
import re
import numpy as np
from scipy.signal import resample

import time


In [51]:
# mac
#sensor_data_folder = '/Users/liuxinqing/Documents/Kfall/sensor_data'  # Update with the path to sensor data
#label_data_folder = '/Users/liuxinqing/Documents/Kfall/label_data'  
# windows 
sensor_data_folder = 'G:\MLonMCU\Kfall_dataset\sensor_data'  # Update with the path to sensor data
label_data_folder = 'G:\MLonMCU\Kfall_dataset\label_data' 
num_data_points = 256

In [86]:
def process_data(sensor_folder, label_file, num_data_points):
        # 100fps
        num_data_points = num_data_points

        #print(label)
        label_data = pd.read_excel(label_file)

        fall_file = {'file_name' : [], 'Fall_onset_frame' : [], 'Fall_impact_frame' : []}
        #print(task_code)
        # Iterate over each row in the label file
        for col, row in label_data.iterrows():
            #print(col, row)
            if not pd.isna(row['Task Code (Task ID)']):

                #print(row['Task Code (Task ID)'])
                match = re.search(r'\((.*?)\)', str(row['Task Code (Task ID)']))
                if match:
                    task_id = match.group(1)
                    #print(task_id)
                idx = 0
                
                while True:
                    if col+idx+1 == len(label_data) or not pd.isna(label_data.iloc[col+idx+1, 0]):
                        trial_id = label_data.iloc[col+idx, 2]
                        file_name = f'S06T{task_id}R0{trial_id}.csv'
                        Fall_onset_frame = label_data.iloc[col+idx, 3]
                        Fall_impact_frame = label_data.iloc[col+idx, 4]
                        fall_file['file_name'].append(file_name)
                        fall_file['Fall_onset_frame'].append(Fall_onset_frame)
                        fall_file['Fall_impact_frame'].append(Fall_impact_frame)
                        #print(file_name, Fall_onset_frame, Fall_impact_frame)
                        break
                    trial_id = label_data.iloc[col+idx, 2]
                    file_name = f'S06T{task_id}R0{trial_id}.csv'
                    Fall_onset_frame = label_data.iloc[col+idx, 3]
                    Fall_impact_frame = label_data.iloc[col+idx, 4]
                    fall_file['file_name'].append(file_name)
                    fall_file['Fall_onset_frame'].append(Fall_onset_frame)
                    fall_file['Fall_impact_frame'].append(Fall_impact_frame)
                    idx += 1


        #num_files = len(os.listdir(sensor_folder))
        # create a np array to store the data, the shape is (num_files, not known yet, not known yet)

        processed_data = []
        processed_label = []

        #iterare over each file in the sensor data folder
        for file in os.listdir(sensor_folder):

            if file in fall_file['file_name']:

                # give the mathcing fall data file name
                fall_data = fall_file['file_name'][fall_file['file_name'].index(file)]
                # open the fall data file
                data_file = os.path.join(sensor_folder, fall_data)
                #data = load_data(data_file)
                data = pd.read_csv(data_file)
                # drop the column of TimeStamp(s)
                data = data.drop(columns=['TimeStamp(s)'])
                data['label'] = 0
   
                Fall_onset_frame = fall_file['Fall_onset_frame'][fall_file['file_name'].index(file)]
                Fall_impact_frame = fall_file['Fall_impact_frame'][fall_file['file_name'].index(file)]

           
                # label: 0 for before fall, 1 for pre-impact fall, 2 for pos-impact fall
                # add a column of label to the first column
                # frame count smaller than Fall_onset_frame is labeled as 0,
                # frame count between Fall_onset_frame and Fall_impact_frame is labeled as 1,
                # frame count between Fall_impact_frame and (Fall_impact_frame+50) is labeled as 2,
                # frame count larger than (Fall_impact_frame+50) removed for labeling
                # Label the rows as 'pre-impact fall'
                data.loc[(data['FrameCounter'] >= Fall_onset_frame) & (data['FrameCounter'] < Fall_impact_frame), 'label'] = 1

                # Label the rows as 'post-impact fall'
                data.loc[(data['FrameCounter'] >= Fall_impact_frame) & (data['FrameCounter'] <= Fall_impact_frame + 50), 'label'] = 2
                # remove the column of FrameCounter
                data = data.drop(columns=['FrameCounter'])

                # TODO: Decide whether to remove the rows where the frame count is larger than Fall_impact_frame + 50
                # data = data[data['FrameCounter'] <= Fall_impact_frame + 50]

                # print the label that is not 0
                #print(data[data['label'] != 0])

                #print(data)


                
            else:
                #print(f"{file} is not a fall data file.")
                data_file = os.path.join(sensor_folder, file)

                data = pd.read_csv(data_file)
                data = data.drop(columns=['TimeStamp(s)'])
                data = data.drop(columns=['FrameCounter'])
                data['label'] = 0

            # check whether the file is among one of the fall data files
            #print(len(data))
            # extract the labal as np array
            label = data['label'].values

            # drop the label column
            data = data.drop(columns=['label'])
            # extract the data as np array
            data = data.values
            processed_data.append(data)
            processed_label.append(label)
            #print(data.shape)
            #print(label.shape)
    

        return processed_data, processed_label # list of np array, list of np array



#label_file = 'G:\MLonMCU\Kfall_dataset\label_data\SA06_label.xlsx'
#sensor_folder = 'G:\MLonMCU\Kfall_dataset\sensor_data\SA06'
#processed_data, processed_label = process_data(sensor_folder, label_file, num_data_points)


In [87]:
def process_all_folder(sensor_data_folder, label_data_folder, num_data_points):
    all_data = []
    all_label = []
    # delete the hidden file .DS_Store
    if '.DS_Store' in os.listdir(sensor_data_folder):
        os.remove(os.path.join(sensor_data_folder, '.DS_Store'))
    
    # get length of the sensor data folder
    num_sensor_data = len(os.listdir(sensor_data_folder))
    count = 0
    # iterate over each file in the sensor data folder and use process_data function to process the data
    for file in os.listdir(sensor_data_folder):
        #print(file)
        print(f"Processing {count}/{num_sensor_data} folder...")

        sensor_folder = os.path.join(sensor_data_folder, file)
        label_file_name = file.split('.')[0] + '_label.xlsx'
        label_file = os.path.join(label_data_folder, label_file_name)
        # open the label file
        #label_file = os.path.join(label_data_folder, label_file_name)

        # load the label file
        #label_file = pd.read_excel(label_file)
        #print(sensor_folder)
        #print(label_file)
        
        processed_data, processed_label = process_data(sensor_folder, label_file, num_data_points)
        all_data.append(processed_data)
        all_label.append(processed_label)
        # print the 
        count += 1
    return all_data, all_label




In [ ]:
data, label = process_all_folder(sensor_data_folder, label_data_folder, num_data_points)
